In [ ]:
import pandas as pd

# Load the dataset
file_path = 'Pakistan Largest Ecommerce Dataset.csv'
data = pd.read_csv(file_path)

print(data.head())

# List of columns to drop
columns_to_drop = ['item_id', ' MV ', 'created_at', 'increment_id', 'sales_commission_code', 'discount_amount', 'payment_method', 'Working Date', 'BI Status', 'Year', 'Month', 'Customer Since', 'M-Y', 'FY', "Unnamed: 21", "Unnamed: 22", "Unnamed: 23", "Unnamed: 24", "Unnamed: 25"]
data.drop(columns=columns_to_drop, inplace=True)

print("____________________________________________________")
print(data.head())

<ipython-input-1-61ec3b8fdbdf>:5: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


    item_id          status created_at  \
0  211131.0        complete   7/1/2016   
1  211133.0        canceled   7/1/2016   
2  211134.0        canceled   7/1/2016   
3  211135.0        complete   7/1/2016   
4  211136.0  order_refunded   7/1/2016   

                                                 sku   price  qty_ordered  \
0                                  kreations_YI 06-L  1950.0          1.0   
1  kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...   240.0          1.0   
2                                 Ego_UP0017-999-MR0  2450.0          1.0   
3                                     kcc_krone deal   360.0          1.0   
4                                        BK7010400AG   555.0          2.0   

   grand_total increment_id    category_name_1 sales_commission_code  ...  \
0       1950.0    100147443    Women's Fashion                    \N  ...   
1        240.0    100147444  Beauty & Grooming                    \N  ...   
2       2450.0    100147445    Women's Fashion       

In [ ]:
# Duomenu analize
print(data.isnull().sum())
print("__________________________")

data.dropna(inplace=True)
print(data.isnull().sum())

print("__________________________")

num_records = data.shape[0]
print(f"Number of records in the DataFrame: {num_records}")


data['Customer ID'] = data['Customer ID'].astype('int32')
data['price'] = data['price'].astype('float32')
data['qty_ordered'] = data['qty_ordered'].astype('float32')
data['grand_total'] = data['grand_total'].astype('float32')
data['status'] = data['status'].astype('category')
data['sku'] = data['sku'].astype('object')
data['price'] = data['price'].astype('float32')
data['category_name_1'] = data['category_name_1'].astype('category')

status             464066
sku                464071
price              464051
qty_ordered        464051
grand_total        464051
category_name_1    464215
Customer ID        464062
dtype: int64
__________________________
status             0
sku                0
price              0
qty_ordered        0
grand_total        0
category_name_1    0
Customer ID        0
dtype: int64
__________________________
Number of records in the DataFrame: 584314


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Count the number of orders per "Customer ID"
order_counts = data['Customer ID'].value_counts()
print("Number of orders per Customer ID:")
print(order_counts)

# Count the number of orders per "sku (Product name)"
sku_counts = data['sku'].value_counts()
print("Number of orders per sku:")
print(sku_counts)

# Count the number of orders per "status"
status_counts = data['status'].value_counts()
print("Number of orders per status:")
print(status_counts)

cleaned_data_path = 'cleaned_ecommerce_data.csv'
data.to_csv(cleaned_data_path, index=False)

Number of orders per Customer ID:
Customer ID
85775     2524
163       2349
35        1877
33        1397
31025     1369
          ... 
52111        1
52114        1
52116        1
52118        1
115326       1
Name: count, Length: 115304, dtype: int64
Number of orders per sku:
sku
MATSAM59DB75ADB2F80              3775
Al Muhafiz Sohan Halwa Almond    2258
emart_00-7                       2027
kcc_krone deal                   1894
infinix_Zero 4-Grey              1793
                                 ... 
WOFBAT5A0144DB102B3-37              1
Essential_003-M                     1
BAGRAS59AE3485778F2                 1
BAGULR59E84898A97E1                 1
WOFSCE5AE00357AECDE                 1
Name: count, Length: 84869, dtype: int64
Number of orders per sku:
status
complete          233684
canceled          201127
received           77284
order_refunded     59498
refund              8020
cod                 2854
paid                1159
closed               494
payment_review        57


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
import joblib
import numpy as np
from tqdm import tqdm

# Define data types
dtypes = {
    'status': 'category',
    'sku': 'object',
    'price': 'float32',
    'qty_ordered': 'float32',
    'grand_total': 'float32',
    'category_name_1': 'category',
    'Customer ID': 'int32'
}

# Load your dataset in chunks
file_path = 'cleaned_ecommerce_data.csv'
chunk_size = 10000

# Initialize encoders
encoder_sku = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
encoder_category = OneHotEncoder(sparse_output=True, handle_unknown='ignore')

all_labels = set()

# Process data in chunks and collect all unique labels
for chunk in pd.read_csv(file_path, chunksize=chunk_size, dtype=dtypes):
    # Remove rows with missing SKUs or Customer IDs
    chunk = chunk.dropna(subset=['sku', 'Customer ID'])

    # Transform 'status' to a binary indicator
    chunk['status'] = chunk['status'].apply(lambda x: 1 if x in ['complete', 'received'] else 0)

    # Filter only completed purchases for training the next SKU model
    chunk = chunk[chunk['status'] == 1].copy()

    # Create target variable (next SKU to buy)
    chunk['next_sku'] = chunk.groupby('Customer ID')['sku'].shift(-1)

    # Drop rows with missing target variable
    chunk = chunk.dropna(subset=['next_sku'])

    # Collect all unique labels
    all_labels.update(chunk['next_sku'].unique())

    # Fit encoders on the current chunk
    encoder_sku.fit(chunk[['sku']])
    encoder_category.fit(chunk[['category_name_1']])

# Save the fitted encoders
joblib.dump(encoder_sku, 'encoder_sku.pkl')
joblib.dump(encoder_category, 'encoder_category.pkl')

# Save all collected labels
joblib.dump(list(all_labels), 'all_labels.pkl')

# Incremental Model Training
# Load the processed data in chunks again
data_iterator = pd.read_csv(file_path, chunksize=chunk_size, dtype=dtypes)

# Initialize the model
model = SGDClassifier(random_state=42)

# Initialize the label encoder with all labels
label_encoder = LabelEncoder()
label_encoder.fit(list(all_labels))

# Perform incremental training with a progress bar
for chunk in tqdm(data_iterator, desc="Training Progress"):
    # Remove rows with missing SKUs or Customer IDs
    chunk = chunk.dropna(subset=['sku', 'Customer ID'])

    # Transform 'status' to a binary indicator
    chunk['status'] = chunk['status'].apply(lambda x: 1 if x in ['complete', 'received'] else 0)

    # Filter only completed purchases for training the next SKU model
    chunk = chunk[chunk['status'] == 1].copy()

    # Create target variable (next SKU to buy)
    chunk['next_sku'] = chunk.groupby('Customer ID')['sku'].shift(-1)

    # Drop rows with missing target variable
    chunk = chunk.dropna(subset=['next_sku'])

    # One-hot encode the current chunk
    encoded_sku = encoder_sku.transform(chunk[['sku']])
    encoded_category = encoder_category.transform(chunk[['category_name_1']])

    # Combine the original data with the encoded features
    chunk_encoded = pd.concat([chunk.reset_index(drop=True),
                               pd.DataFrame(encoded_sku.toarray(), columns=encoder_sku.get_feature_names_out(['sku'])),
                               pd.DataFrame(encoded_category.toarray(), columns=encoder_category.get_feature_names_out(['category_name_1']))], axis=1)

    # Drop original 'sku' and 'category_name_1' columns and 'status'
    chunk_encoded = chunk_encoded.drop(columns=['sku', 'category_name_1', 'status'])

    # Define features and target
    X_chunk = chunk_encoded.drop(columns=['next_sku'])
    y_chunk = chunk_encoded['next_sku']

    # Transform the target variable
    y_chunk_encoded = label_encoder.transform(y_chunk)

    # Perform partial fit
    model.partial_fit(X_chunk, y_chunk_encoded, classes=label_encoder.transform(label_encoder.classes_))

# Save the trained model and label encoder
joblib.dump(model, 'sku_prediction_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')


Training Progress: 59it [2:18:39, 141.01s/it]


['label_encoder.pkl']

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack, csr_matrix
import numpy as np
import joblib

# Load the fitted encoders and model
encoder_sku = joblib.load('encoder_sku.pkl')
encoder_category = joblib.load('encoder_category.pkl')
model = joblib.load('sku_prediction_model.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Define data types
dtypes = {
    'status': 'category',
    'sku': 'object',
    'price': 'float32',
    'qty_ordered': 'float32',
    'grand_total': 'float32',
    'category_name_1': 'category',
    'Customer ID': 'int32'  # Or 'object' if Customer ID includes non-numeric values
}

# Load the full dataset with specified data types
file_path = 'cleaned_ecommerce_data.csv'  # Update this path to your dataset location
data = pd.read_csv(file_path, dtype=dtypes)

# Remove rows with missing SKUs or Customer IDs
data = data.dropna(subset=['sku', 'Customer ID'])

# Transform 'status' to a binary indicator
data['status'] = data['status'].apply(lambda x: 1 if x in ['complete', 'received'] else 0)

# Function to get the next SKU prediction for a specific user
def user_next_sku_prediction(user_id):
    # Filter data for the specific user
    user_data = data[data['Customer ID'] == user_id]

    if user_data.empty:
        print("User ID not found in the dataset.")
        return

    # Display what the user has already bought
    print(f"Purchase history for user {user_id}:")
    print(user_data[['sku', 'category_name_1', 'qty_ordered', 'price', 'grand_total']])

    # Prepare the data for prediction
    encoded_sku = encoder_sku.transform(user_data[['sku']])
    encoded_category = encoder_category.transform(user_data[['category_name_1']])

    # Combine encoded features
    encoded_features = hstack([encoded_sku, encoded_category])

    # Aggregate encoded features
    encoded_features_agg = encoded_features.sum(axis=0)

    # Aggregate user data
    user_agg_data = user_data.groupby('Customer ID').agg({
        'price': 'sum',
        'qty_ordered': 'sum',
        'grand_total': 'sum'
    }).reset_index()

    # Combine encoded features with user aggregated data
    agg_values = user_agg_data.drop(columns=['Customer ID']).values
    agg_values_csr = csr_matrix(agg_values)
    final_features = hstack([encoded_features_agg, agg_values_csr])

    # Convert to CSR format for subscriptable operations
    final_features = final_features.tocsr()

    # Ensure the final feature dimensions match
    num_features_model = model.n_features_in_
    if final_features.shape[1] != num_features_model:
        if final_features.shape[1] < num_features_model:
            # Add zero columns to match dimensions
            zero_padding = csr_matrix((final_features.shape[0], num_features_model - final_features.shape[1]))
            final_features = hstack([final_features, zero_padding])
        elif final_features.shape[1] > num_features_model:
            # Truncate columns to match dimensions
            final_features = final_features[:, :num_features_model]

    # Predict the next SKU
    next_sku_pred = model.predict(final_features)
    next_sku = label_encoder.inverse_transform(next_sku_pred)

    print(f"\nPredicted next item for user {user_id}: {next_sku[0]}")

# Example usage
user_id = 35  # Replace with an actual user ID from your dataset
user_next_sku_prediction(user_id)


Purchase history for user 35:
                        sku    category_name_1  qty_ordered        price  \
59            itter_AB 1214  Beauty & Grooming          1.0   300.000000   
67            ajmery_F9-981      Men's Fashion          1.0   999.000000   
70           kcc_krone deal  Beauty & Grooming          1.0   360.000000   
71      RS_Habshi Halwa Tin            Soghaat          1.0   280.000000   
262              Haier M106  Mobiles & Tablets          1.0  2490.000000   
...                     ...                ...          ...          ...   
121732          stinnos_945        Kids & Baby          1.0   940.000000   
122442      Essential_006-L      Men's Fashion          1.0   425.000000   
122444      Essential_005-L      Men's Fashion          1.0   425.000000   
122446     Aybeez_ABZ-277-L      Men's Fashion          1.0   439.600006   
122507      Essential_005-M      Men's Fashion          1.0   425.000000   

        grand_total  
59       300.000000  
67       999.

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(
